## 가상현실RPG게임 라플라스의 악마는 날씨가 큰 변수를 차지하는 게임이다. 게임 밸런스 조정을 위해 한달 간의 날씨 데이터를 분석하려고 한다.

Laplace's Demon.csv

|컬럼|정의|type|
|:---|:---|:---:|
|date|날짜|object|
|area|지역명|object|
|temp|온도|float64|
|atemp|체감온도|float64|
|humidity|습도|float64|
|windspeed|풍속|float64|
|rain|날씨(값이 0이면 맑음, 1이면 비가 내림)|int64|  

정답 및 해설 : https://tjd229.tistory.com/24

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('../content/tjd229/Laplace\'s Demon.csv')
df.shape

(992, 7)

### Q1. 체감온도(atemp)가 정규분포와의 관계를 분석하려 한다. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.

단계 1 : 체감온도(atemp)가 0보다 큰 데이터를 Train Set으로, 체감온도(atemp)가 0이하인 데이터를 Test Set으로 분할한다.  
단계 2 : Train Set을 이용하여 LinearRegression을 학습하고, Test Set에 적용한다. 
- 독립 변수(총 4개) : 온도(temp), 습도(humidity), 풍속(windspeed), 날씨(rain)
- 종속 변수 : 체감온도(atemp)
        
단계 3 : Train Set에서 체감온도(atemp)의 평균과 표준편차를 구하고, 해당 평균과 표준편차를 가지는 정규 분포를 구한다.

단계 4 : 단계 2에서 구한 Test Set의 예측값 $ \hat{y_i} $와 단계 3에서 구한 정규 분포의 0.25 분위수에 해당하는 값 $ y $에 대하여 아래 정의된 Measure M을 계산한 값은?  
$$ M = \biggl(\frac{1}{n} \sum_{i=1}^{n}{(y - \hat{y_i})^2}\biggr)^{\frac{1}{2}}, \quad \hat{y_i}: 예측값, y: 실제값 $$


※ 정답은 반올림하여 소수점 둘째 자리까지 출력하시오.
(정답 예시: 2.29)

In [2]:
from sklearn.linear_model import LinearRegression  
from scipy.stats import norm 
df1 = df.copy()

In [3]:
X_train = df1.loc[df1.atemp > 0, ['temp','humidity','windspeed','rain']]
X_test = df1.loc[df1.atemp <= 0, ['temp','humidity','windspeed','rain']]
y_train = df1.loc[df1.atemp > 0, 'atemp']
y_test = df1.loc[df1.atemp <= 0, 'atemp']

In [4]:
my_LR = LinearRegression()
my_LR.fit(X_train, y_train)
y_pred = my_LR.predict(X_test)

In [5]:
df1.loc[df1.atemp > 0,'atemp'].mean(), df1.loc[df1.atemp > 0,'atemp'].std()

(46.39257288379459, 34.36297082966668)

👉 norm.ppf  

norm : 정규분포 함수   norm(기대값,표준편차)  
pmf : 확률질량함수(probability mass function)  
pdf : 확률밀도함수(probability density function)  
cdf : 누적분포함수(cumulative distribution function)  
ppf : 누적분포함수의 역함수(inverse cumulative distribution function)  

https://datascienceschool.net/02%20mathematics/08.01%20%EC%82%AC%EC%9D%B4%ED%8C%8C%EC%9D%B4%EB%A5%BC%20%EC%9D%B4%EC%9A%A9%ED%95%9C%20%ED%99%95%EB%A5%A0%EB%B6%84%ED%8F%AC%20%EB%B6%84%EC%84%9D.html

In [6]:

my_norm= norm(df1.loc[df1.atemp > 0,'atemp'].mean(),df1.loc[df1.atemp > 0,'atemp'].std())
y_test2= my_norm.ppf(0.25)

In [7]:
(((y_pred - y_test2)**2).mean())**0.5

13.56365578009182

RMSE  
y_test 는 리스트 이지만, y_test2 정수 이므로,   
동일한 크기의 리스트에 y_test2 값을 모두 넣어야 RMSE 가 계산가능하다.

In [8]:
y_test3 = y_test.copy()
y_test3[:] = y_test2

In [9]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test3, y_pred, squared=False)


13.56365578009182

#### 필요 라이브러리 함수,클래스 및 설정값 목록  


from sklearn.linear_model import LinearRegression  
from scipy.stats import norm  
문제 지시 외 Default 값 사용  

### Q2. 지역별 강수량을 확인하려 한다.  다음 단계에 따라 분석을 수행하고 질문에 답하시오.


단계 1 : 날씨(rain) 컬럼에서 값이 0인 데이터를 -1로 바꾼다.

단계 2 : 32종류의 area별로, 아래의 단계 3을 수행하여, cum_precipitation 컬럼의 최대값과 최소값 차이가 가장 큰 값을 구하시오 

단계 3 : 각 area별로 날짜순으로 정렬 후, 아래 규칙에 따라 cum_precipitation 컬럼을 생성한다.  
- $z$를 cum_precipitation, $ x $를 rain이라 할 때, $ z_1 = x_1 $ 이고,  
- j>1에 대하여, $ z_j = \sum_{i=1}^{j}{x_i} $ 이다.  



In [10]:
df2 = df.copy()

In [33]:
df2['rain'] = df2.rain.replace(0,-1)

In [17]:
df2.head()

,date,area,temp,atemp,humidity,windspeed,rain
0,2023-01-13,Cerulean City,26.869606,-8.894446,43.138450,6.009597,1
1,2023-01-17,Cerulean City,18.385474,-4.988309,88.427747,20.864357,1
2,2023-01-31,Seafoam Islands,26.867670,0.659525,30.940357,30.733466,1
3,2023-01-31,Goldenrod City,24.329853,107.903405,64.554282,8.618055,-1
4,2023-01-23,Cianwood City,30.726903,75.236753,51.298311,13.717975,-1


In [34]:
areas = df2.area.unique()

👉 groupby 대신 slicing (df2_0)

In [36]:
df2_0 = df2.copy()
MM_list=[]

for area_indx in areas:
    temp_df = df2_0.loc[df2_0.area == area_indx,:]                  # slicing df 에서 
    temp_list = temp_df.sort_values(by='date')['rain'].cumsum()     # date sort 한 rain의 cumsum 값 리스트 
    df2_0.loc[df2_0.area == area_indx,'result'] = temp_list         # 을 원래 df2_0 의 result 에 입력
    MM_list.append(temp_list.max()-temp_list.min())                 # 하고, 리스트 max() - min() 값 저장

pd.DataFrame( MM_list,
             index=areas,
             columns=['max-min'])

,max-min
Cerulean City,8
Seafoam Islands,6
Goldenrod City,6
Cianwood City,8
Mt. Mortar,11
Olivine City,9
Viridian Forest,9
Mt. Moon,10
Mahogany Town,6
Ruins of Alph,6


In [41]:
df2_0.loc[df.area == 'Lake of Rage','result'].max() - df2_0.loc[df.area == 'Lake of Rage','result'].min()

18.0

groupby 이용 (df2_1)

In [43]:
df2_1 = df2.groupby(['area','date'])['rain'].sum().reset_index().copy()
df2_1.head()

# groupby 로 인해 sort 된듯

,area,date,rain
0,Azalea Town,2023-01-01,-1
1,Azalea Town,2023-01-02,1
2,Azalea Town,2023-01-03,-1
3,Azalea Town,2023-01-04,-1
4,Azalea Town,2023-01-05,-1


In [22]:
cumsum_df = pd.DataFrame()
MM_list =[]

for area_indx in areas:
    cumsum_df[area_indx] = df2_1.loc[df2_1.area == area_indx,'rain'].reset_index(drop=True).cumsum()
    MM_list.append(cumsum_df[area_indx].max() - cumsum_df[area_indx].min())


In [23]:

pd.DataFrame( MM_list,
            index=areas,
            columns=['max_min'])

,max_min
Cerulean City,8
Seafoam Islands,6
Goldenrod City,6
Cianwood City,8
Mt. Mortar,11
Olivine City,9
Viridian Forest,9
Mt. Moon,10
Mahogany Town,6
Ruins of Alph,6


In [153]:

result_list=[]

for area_indx in areas:
    temp_df= df2_1.loc[df2_1.area == area_indx,].copy()
    temp_df['cum_precipitation'] = temp_df['rain'].cumsum()
    delta = temp_df['cum_precipitation'].max()-temp_df['cum_precipitation'].min()
    result_list.append([area_indx,delta])

result_df = pd.DataFrame(result_list,
                         columns=['Area','max_min'])

result_df



,Area,max_min
0,Azalea Town,8
1,Blackthorn City,10
2,Celadon City,16
3,Cerulean City,8
4,Cherrygrove City,9
5,Cianwood City,8
6,Cinnabar Island,12
7,Ecruteak City,10
8,Fuchsia City,6
9,Goldenrod City,6


cumsum()  

뭔가 apply, itertuples 로 가능할것 같은데.  
apply, itertuples 는 한 row 씩 계산하는 거라. cumsum 을 만들어야 함.


### Q3. 날씨에 따른 게임 밸런스를 확인하기 위해 군집화를 하려 한다. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.


단계 1 : 온도, 체감온도, 습도, 풍속을 Z-score 표준화(Standardization) 한다.  
단계 2 : 독립 변수들에 대해 K-means 군집 분석을 수행한다. 이 때, 군집 수는 2개로 한다.  
- 독립 변수(총 5개) : 온도(temp), 체감온도(atemp), 습도(humidity), 풍속(windspeed), 날씨(rain)  

단계 3 : 군집 별 체감온도(atemp) 평균을 구하고 두 평균을 모두 더한 값을 구하시오  
            
※ 정답은 내림하여 소수점 둘째 자리까지 출력하시오.
(정답 예시: 2.29)

In [1]:
from sklearn.cluster import KMeans  
from sklearn.preprocessing import StandardScaler  

#### 필요 라이브러리 함수,클래스 및 설정값 목록  


from sklearn.cluster import KMeans  
from sklearn.preprocessing import StandardScaler  
random_state=229  
문제 지시 외 Default 값 사용  